In [ ]:
! pip install bs4
! pip install urllib
! pip install selenium
! pip install csv

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
import csv

base_url = 'https://virtual-youtuber.userlocal.jp'
sub_url = '/document/ranking'
date = datetime.now().strftime('%Y%m%d')
filename = f'[{date}] vtuber_all_ranking.csv'
vtubers = []

# Chrome 브라우저 열기 (작동 후 10초 로딩 여유시간 확보)
driver = webdriver.Chrome('C:\Program Files\chromedriver.exe')
driver.implicitly_wait(10)

while True:
    # Vtuber 정보 크롤링
    driver.get(base_url + sub_url)
    page = BeautifulSoup(driver.page_source, 'html.parser')
    vtuber_info = page.find_all('tr')

    for vtuber in vtuber_info:
        ranking = vtuber.find('strong').get_text()[:-1]
        name_agency = vtuber.find_all('a')
        jp_name = name_agency[1].get_text().strip()
        if vtuber.find('span', class_ = 'text-secondary') is not None:
            eng_name = vtuber.find('span', class_ = 'text-secondary').get_text().strip()
        else:
            eng_name = ' '  
        if len(name_agency) >= 5:
            agency = name_agency[2].get_text().strip()
        else:
            agency = ' '
        total_subs = vtuber.find('span', class_ = 'text-success font-weight-bold').get_text().strip()[:-2]
        total_views = vtuber.find('span', class_ = 'text-danger font-weight-bold').get_text().strip()[:-2]

        vtubers.append({'ranking':ranking, 'jp_name':jp_name, 'eng_name':eng_name, 'agency':agency, 'total_subs':total_subs, 'total_views':total_views})

    # 다음 페이지 호출
    possible_pages = page.find('ul', class_ = 'pagination justify-content-center')
    next_page = possible_pages.find('a', {'rel':'next'})
    if next_page is not None:
        sub_url = next_page.get('href')
    else:
        break

# CSV파일로 출력
with open(filename, 'w', encoding = 'utf-8') as csvfile:
    fieldnames = ['ranking', 'jp_name', 'eng_name', 'agency', 'total_subs', 'total_views']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for vtuber in vtubers:
        writer.writerow(vtuber)
    else:
        print(f'======= TOTAL OF {len(vtubers)} VTUBERS RECORDED! =======')

# 누락된 랭킹 확인
with open(filename, encoding = 'utf-8') as csvfile:
    data = csv.DictReader(csvfile, delimiter=',')
    ranks = list(range(1,2001))
    
    print('Missing ranks:', end = ' ')
    for row in data:
        rank = int(row['ranking'])
        if rank not in ranks:
            print(f'Rank {rank} is missing!', end = ' ')